In [ ]:
import os
import ipywidgets as widgets
from openbb_terminal import OpenBBFigure, theme

import yfinance as yf
from IPython.display import display

In [ ]:
opts = [
    "lastTradeDate",
    "strike",
    "lastPrice",
    "bid",
    "ask",
    "change",
    "percentChange",
    "volume",
    "openInterest",
    "impliedVolatility",
]


def clean_str(string):
    new_str = ""
    for letter in string:
        if letter.isupper():
            new_str += " "
        new_str += letter
    return new_str.title()


def format_plotly(fig: OpenBBFigure, x, y, ticker, expiry, inst):
    fig.update_yaxes(title=clean_str(y))
    fig.update_xaxes(title=clean_str(x))
    expires = ", ".join(expiry)
    title = (
        f"{clean_str(y)} vs. {clean_str(x)} for {ticker.upper()} {inst}s on {expires}"
    )
    fig.update_layout(
        margin=dict(t=40),
        autosize=False,
        width=1000,
        height=500,
        title=dict(
            text=title,
            y=0.98,
            x=0.5,
            xanchor="center",
            yanchor="top",
        ),
    )

In [ ]:
class Chart:
    def __init__(self):
        self.last_ticker = ""
        self.dfs = {}

    def update(self, ticker, expiry, inst, x, y):
        if ticker:
            stock = yf.Ticker(ticker)
            if ticker != self.last_ticker:
                exp_w.options = stock.options
                self.last_ticker = ticker

            if expiry:
                fig = OpenBBFigure()
                for expire in expiry:
                    if expire not in self.dfs:
                        self.dfs[expire] = stock.option_chain(expire)
                    group = self.dfs[expire]
                    df = group.calls if inst == "Call" else group.puts

                    fig.add_scatter(
                        x=df[x],
                        y=df[y],
                        mode="lines",
                        connectgaps=True,
                        name=expire,
                    )

                format_plotly(fig, x, y, ticker, expiry, inst)

                if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
                    fig.show(
                        config={"showTips": False, "displaylogo": False},
                        renderer="notebook",
                    )
                else:
                    fig.show(config={"showTips": False, "displaylogo": False})


x_w = widgets.Dropdown(options=opts, value="strike", description="X")
y_w = widgets.Dropdown(options=opts, value="lastPrice", description="Y")
tickers_w = widgets.Text(description="Ticker")
exp_w = widgets.SelectMultiple(description="Expiry")
inst_w = widgets.Dropdown(options=["Put", "Call"], value="Call", description="Type")

select_opt = widgets.VBox([tickers_w, exp_w])
select_plot = widgets.VBox([inst_w, x_w, y_w])
controls = widgets.HBox([select_opt, select_plot])

chart = Chart()
stocks_view = widgets.interactive_output(
    chart.update,
    {"ticker": tickers_w, "expiry": exp_w, "inst": inst_w, "x": x_w, "y": y_w},
)

title_html = "<h1>Option Chain Dashboard</h1>"
app_contents = [
    widgets.HTML(title_html),
    controls,
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)